In [1]:
import os
import json
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

# ---------- OpenAI ----------
client = OpenAI()
MODEL = "qwen/qwen3-next-80b-a3b-instruct"
print("DEBUG: API Loaded:", bool(os.getenv("OPENAI_API_KEY")))

# ---------- SQLite ----------
DB_PATH = "sample.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
print("DEBUG: SQLite connected:", DB_PATH)

# ---------- Excel ----------
EXCEL_PATH = "mall_data.xlsx"
inventory_df = pd.read_excel(EXCEL_PATH, sheet_name="inventory")
staff_orders_df = pd.read_excel(EXCEL_PATH, sheet_name="staff_orders")
machines_df = pd.read_excel(EXCEL_PATH, sheet_name="machines")
vendor_detail_df = pd.read_excel(EXCEL_PATH, sheet_name="vendor_detail")

print("DEBUG: Excel loaded")
print("DEBUG: inventory rows:", len(inventory_df))
print("DEBUG: staff_orders rows:", len(staff_orders_df))
print("DEBUG: machines rows:", len(machines_df))
print("DEBUG: vendor_detail rows:", len(vendor_detail_df))


DEBUG: API Loaded: True
DEBUG: SQLite connected: sample.db
DEBUG: Excel loaded
DEBUG: inventory rows: 40
DEBUG: staff_orders rows: 20
DEBUG: machines rows: 20
DEBUG: vendor_detail rows: 10


In [2]:
import whisper
import tempfile
import os

# Load Whisper model once
print("DEBUG: Loading Whisper model...")
whisper_model = whisper.load_model("base")
print("DEBUG: Whisper loaded")


def voice_to_text(audio_path):
    """
    Converts recorded audio to text using Whisper
    """
    print("DEBUG: Transcribing audio:", audio_path)
    result = whisper_model.transcribe(audio_path)
    text = result["text"].strip()
    print("DEBUG Voice Text:", text)
    return text


DEBUG: Loading Whisper model...
DEBUG: Whisper loaded


In [3]:
# Create a cursor object to execute SQL commands on the SQLite database
cur = conn.cursor()


# STEP 1: Drop existing tables
# This ensures old data/schema is removed before inserting fresh Excel data

print("DEBUG: Dropping existing tables")
cur.execute("DROP TABLE IF EXISTS inventory")
cur.execute("DROP TABLE IF EXISTS machines")
cur.execute("DROP TABLE IF EXISTS staff_orders")
cur.execute("DROP TABLE IF EXISTS vendor_detail")
conn.commit()

# STEP 2: Write Excel sheets into SQLite tables
# Each pandas DataFrame is converted into a SQLite table

print("DEBUG: Writing Excel → SQLite (correct mapping)")
inventory_df.to_sql("inventory", conn, if_exists="replace", index=False)
machines_df.to_sql("machines", conn, if_exists="replace", index=False)
staff_orders_df.to_sql("staff_orders", conn, if_exists="replace", index=False)
vendor_detail_df.to_sql("vendor_detail", conn, if_exists="replace", index=False)

print("DEBUG: Tables written")


# STEP 3: Verify table schemas
# Prints column names and types for each created table
# Useful for debugging and validation

for t in ["inventory", "machines", "staff_orders", "vendor_detail"]:
    cur.execute(f"PRAGMA table_info({t})")
    print(f"\nDEBUG {t} columns:")
    for row in cur.fetchall():
        print(row)


DEBUG: Dropping existing tables
DEBUG: Writing Excel → SQLite (correct mapping)
DEBUG: Tables written

DEBUG inventory columns:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'item_name', 'TEXT', 0, None, 0)
(2, 'category', 'TEXT', 0, None, 0)
(3, 'quantity', 'INTEGER', 0, None, 0)
(4, 'price', 'INTEGER', 0, None, 0)
(5, 'location', 'TEXT', 0, None, 0)

DEBUG machines columns:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'machine_name', 'TEXT', 0, None, 0)
(2, 'location', 'TEXT', 0, None, 0)
(3, 'status', 'TEXT', 0, None, 0)
(4, 'last_maintenance_date', 'TIMESTAMP', 0, None, 0)
(5, 'next_maintenance_date', 'TIMESTAMP', 0, None, 0)
(6, 'purchase_date', 'TIMESTAMP', 0, None, 0)
(7, 'warranty_until', 'TIMESTAMP', 0, None, 0)
(8, 'capacity', 'TEXT', 0, None, 0)
(9, 'energy_usage_kw', 'REAL', 0, None, 0)
(10, 'supplier_vendor_id', 'TEXT', 0, None, 0)
(11, 'notes', 'TEXT', 0, None, 0)

DEBUG staff_orders columns:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'staff_name', 'TEXT', 0, None, 0)
(2, 'item_id', 'INTEGER', 

In [4]:
# DEBUG: Inspect actual machines table schema
# cur = conn.cursor()
# cur.execute("PRAGMA table_info(machines)")
# cols = cur.fetchall()

# print("DEBUG: machines table columns:")
# for c in cols:
#     print(c)


In [5]:
from rapidfuzz import process, fuzz

# SCHEMA DESCRIPTION
# This text is sent to the LLM so it understands:
# - What tables exist
# - What columns each table has
# - What rules to follow while generating JSON

SCHEMA = """
Tables:
inventory(item_name, category, quantity, price, location)
staff_orders(staff_name, item_id, quantity, order_time)
vendor_detail(vendor_id, vendor_name, contact_person, phone, email, primary_items_supplied)

Rules:
- Output ONLY JSON
- Keys: action, table, columns, filters, group_by, limit
- Use ONLY listed tables & columns
- Do NOT generate SQL
- LIKE means "text contains value"
"""


# FEW-SHOT EXAMPLES
# These examples teach the model how to convert
# natural language questions into structured JSON

FEW_SHOTS = [
    {
        "user": "How many popcorn packets do we have?",
        "json": {
            "action": "select",
            "table": "inventory",
            "columns": ["quantity"],
            "filters": [
                {"column": "item_name", "op": "=", "value": "Popcorn"}
            ],
            "group_by": [],
            "limit": None
        }
    },
    {
        "user": "Show Ayush's staff orders",
        "json": {
            "action": "select",
            "table": "staff_orders",
            "columns": ["item_id", "quantity"],
            "filters": [
                {"column": "staff_name", "op": "=", "value": "Ayush"}
            ],
            "group_by": [],
            "limit": None
        }
    },
    {
        "user": "Who supplies popcorn and nachos?",
        "json": {
            "action": "select",
            "table": "vendor_detail",
            "columns": ["vendor_name", "contact_person", "phone"],
            "filters": [
                {"column": "primary_items_supplied", "op": "LIKE", "value": "Popcorn"}
            ],
            "group_by": [],
            "limit": None
        }
    }
]


# QUERY CLASSIFIER
# Decides whether a user query is database-related
# This helps route queries to DB logic vs chat

def is_db_query(q):
    keywords = [
        # inventory / food items
        "popcorn", "nachos", "sprite", "coke", "coffee",
        "burger", "fries", "momos", "ice cream",

        # inventory concepts
        "inventory", "price", "stock", "quantity", "available",

        # staff orders
        "staff", "order", "orders", "ordered",

        # vendors
        "vendor", "supplier", "supplies", "contact", "phone", "email",

        # generic db intent
        "show", "list", "how many", "who"
    ]
    return any(k in q.lower() for k in keywords)


In [6]:

# Utility function to fetch distinct column values
# from a SQLite table.
# These values are used as CANONICAL values for
# fuzzy matching user inputs.
#The Canon Value Normalization Layer ensures that user inputs are mapped to valid database values using fuzzy matching, preventing mismatches due to typos or variations.
def fetch_column_values(table, column):
    cur = conn.cursor()
    cur.execute(f"SELECT DISTINCT {column} FROM {table}")
    vals = [r[0] for r in cur.fetchall() if r[0] is not None]
    print(f"DEBUG Canonical {table}.{column}:", vals)
    return vals



# Canonical values from database
# Used to normalize user intent values

CANON_ITEMS = fetch_column_values("inventory", "item_name")
CANON_STAFF = fetch_column_values("staff_orders", "staff_name")
CANON_VENDORS = fetch_column_values("vendor_detail", "vendor_name")
CANON_VENDOR_CONTACTS = fetch_column_values("vendor_detail", "contact_person")




# Fuzzy matching helpernormalize_intent_values
# Tries to match user-provided text to known values
# using RapidFuzz scoring

def fuzzy_match(term, choices):
    if not term or not choices:
        return None
    match, score, _ = process.extractOne(
        str(term).lower(), choices, scorer=fuzz.WRatio
    )
    print(f"DEBUG fuzzy '{term}' → '{match}' ({score})")
    return match if score >= 70 else None



# Normalizes intent filter values
# Replaces fuzzy user input with canonical DB values
# before query execution

def normalize_intent_values(intent):
    for f in intent.get("filters", []):

        if f["column"] == "item_name":
            m = fuzzy_match(f["value"], CANON_ITEMS)

        elif f["column"] == "staff_name":
            m = fuzzy_match(f["value"], CANON_STAFF)

        elif f["column"] == "vendor_name":
            m = fuzzy_match(f["value"], CANON_VENDORS)

        elif f["column"] == "contact_person":
            m = fuzzy_match(f["value"], CANON_VENDOR_CONTACTS)

        else:
            m = None

        if m:
            print("DEBUG normalized:", f["value"], "→", m)
            f["value"] = m

    return intent


DEBUG Canonical inventory.item_name: ['Popcorn (Salted)', 'Nachos (Cheese)', 'Coke', 'Sprite', 'Caramel Popcorn', 'Cheese Popcorn', 'Chicken Hotdog', 'Veg Burger', 'Chicken Burger', 'Paneer Tikka Wrap', 'French Fries', 'Samosa (2 pcs)', 'Diet Coke', 'Fanta', 'Mineral Water (500ml)', 'Iced Tea (Lemon)', 'Cold Coffee', 'Hot Coffee', 'Cappuccino', 'KitKat', 'M&Ms', 'Snickers', 'Potato Chips', 'Doritos', 'Sweet Corn Cup', 'Chicken Nuggets', 'Combo (Popcorn + Coke)', 'Combo (Nachos + Sprite)', 'Ice Cream (Vanilla)', 'Ice Cream (Chocolate)', 'Brownie', 'Red Bull', 'Popcorn (Large Mix)', 'Mozzarella Sticks', 'Slushie (Berry)', 'Slushie (Mango)', 'Veg Momos', 'Chicken Momos', 'Paper Cups', 'Napkins']
DEBUG Canonical staff_orders.staff_name: ['Ayush', 'Rahul', 'Sneha', 'Vikram', 'Pooja', 'Amit', 'Priya', 'Rohan']
DEBUG Canonical vendor_detail.vendor_name: ['SnackMaster Inc.', 'BevCorp India', 'Coffee Elite', 'Frosty Foods', 'Candy World', 'Hot Bites', 'Wrap & Roll Co.', 'SupplyPro', 'Slush King

In [7]:

# Required keys that MUST be present in the intent JSON
# This is used to validate LLM output structure

REQUIRED_KEYS = {"action", "table", "columns", "filters", "group_by", "limit"}



# Converts a natural language user query into
# a structured intent (JSON) using the LLM

def query_to_intent(user_query):

    # Start prompt with database schema description
    # This tells the LLM what tables and columns exist
    prompt = SCHEMA + "\n\n"

    # Add few-shot examples to guide the LLM
    # These examples teach the expected input → output format
    for ex in FEW_SHOTS:
        prompt += (
            f"User: {ex['user']}\n"
            f"Output: {json.dumps(ex['json'])}\n\n"
        )

    # Append the actual user query
    # Explicitly instruct the model to output ONLY JSON
    prompt += f"User: {user_query}\nOutput ONLY JSON."

    print("DEBUG: Sending to LLM-A")

    # Send prompt to the LLM
    # Temperature = 0 ensures deterministic output
    res = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # Extract raw text output and clean code fences if present
    raw = res.choices[0].message.content.replace("```", "").strip()
    print("DEBUG RAW INTENT:", raw)

    try:
        # Parse LLM output as JSON
        intent = json.loads(raw)

        # Validate that all required keys are present
        if REQUIRED_KEYS - intent.keys():
            raise ValueError("Missing keys")

    except Exception as e:
        # If parsing or validation fails, return unknown intent
        print("ERROR intent parse:", e)
        return {"action": "unknown"}

    # Normalize intent values using fuzzy matching
    # (e.g., item names, staff names, vendor names)
    intent = normalize_intent_values(intent)

    print("DEBUG FINAL INTENT:", intent)
    return intent


In [8]:
# -------------------------------------------------
# Mapping of table names to their corresponding
# pandas DataFrames loaded from Excel.
# This allows query execution directly on DataFrames
# instead of SQLite for fast lookups.
# -------------------------------------------------
DATA_TABLES = {
    "inventory": inventory_df,
    "staff_orders": staff_orders_df,
    "vendor_detail": vendor_detail_df
}


# -------------------------------------------------
# Executes a structured intent on the Excel data
# using pandas operations.
# Supports:
# - table selection
# - equality-based filters
# - column projection
# -------------------------------------------------
def run_excel_query(intent):
    print("DEBUG running query:", intent)

    df = DATA_TABLES.get(intent["table"])
    if df is None:
        return []

    result = df.copy()

    for f in intent["filters"]:
        col = f["column"]
        val = str(f["value"])

        if f["op"] == "=":
            result = result[result[col] == f["value"]]

        elif f["op"] == "LIKE":
            print(f"DEBUG applying LIKE filter: {col} contains '{val}'")
            result = result[result[col].str.contains(val, case=False, na=False)]

    if intent["columns"] != ["*"]:
        result = result[intent["columns"]]

    print("DEBUG result rows:", len(result))
    return result.values.tolist()


In [9]:
# -------------------------------------------------
# Fallback actions for each table
# Used when a query returns no data
# -------------------------------------------------
fallback_actions = {
    "inventory": "Check storage room or food counter.",
    "staff_orders": "Check manual logbook or HR.",
    "vendor_detail": "Contact procurement team or vendor records.",
    "default": "Information not available."
}


# -------------------------------------------------
# Evaluates the query result
# If no data is returned, signals a fallback
# -------------------------------------------------
def result_evaluator(result):
    if not result:
        return {"status": "fallback", "reason": "no_data"}
    return {"status": "ok"}


# -------------------------------------------------
# Generates a human-readable fallback message
# based on the table involved in the intent
# -------------------------------------------------
def get_fallback_message(intent, reason):
    return (
        f"Fallback ({reason}): "
        + fallback_actions.get(intent["table"], fallback_actions["default"])
    )


In [10]:
# -------------------------------------------------
# System prompt for the response-generation LLM
# This LLM explains results in natural language
# and provides guidance when data is missing
# -------------------------------------------------
SYSTEM_PREFIX = """
You are a Mall Operations Assistant.

You have TWO modes of behavior:
Rules:
- Explain ONLY using the Database Result provided
- Do NOT add assumptions, expertise, delivery times, or suggestions
- Do NOT invent vendor capabilities
- If data is missing, clearly say it is missing

For casual chat (jokes, greetings):
- Respond naturally
1. If the user is asking about mall operations
   (inventory, staff orders, machines, vendors):
   - Explain results clearly
   - Use only provided data
   - Do NOT invent information
   - If an item is unavailable, suggest the relevant vendor
     and share contact details if available

2. If the user is asking a casual or general question
   (jokes, greetings, small talk):
   - Respond naturally and helpfully like a normal chatbot
   - You MAY tell jokes or have light conversation
   - Do NOT force mall data into the response

Tone:
- Friendly
- Clear
- Helpful
"""


In [11]:
# -------------------------------------------------
# Sends a message to the explanation LLM (LLM-B)
# Uses conversation history as context
# -------------------------------------------------
def chat_llm(msg, history=None):
    print("DEBUG: Sending to LLM-B")

    messages = [
        {"role": "system", "content": SYSTEM_PREFIX}
    ]

    # Add previous conversation turns (context)
    if history:
        messages.extend(history)

    # Current user message
    messages.append({"role": "user", "content": msg})

    res = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.6
    )

    return res.choices[0].message.content


In [12]:
# -------------------------------------------------
# Formats database query results into a structured
# explanation request for the LLM
# Includes intent so the LLM understands context
# -------------------------------------------------
def format_db_results(user_query, result, intent):
    return f"""
User Query:
{user_query}

Query Intent:
{intent}

Database Result:
{result}

Instructions:
- Explain the result clearly
- If result is empty or indicates unavailability,
  recommend the appropriate vendor and contact details
- Do NOT invent missing data
"""


In [ ]:
DEBUG_LOGS = []

def debug(msg):
    log = f"[DEBUG] {msg}"
    print(log)
    DEBUG_LOGS.append(log)

    if len(DEBUG_LOGS) > 50:
        DEBUG_LOGS.pop(0)


In [13]:
# NOTE:
# Identity and casual questions are handled BEFORE database routing
# to prevent accidental data access and hallucinated responses.
# This ensures non-operational queries never touch inventory logic.

def ask(user_query):
    print("\nDEBUG USER QUERY:", user_query)

    q = user_query.lower().strip()

    # ---- FIX: identity / system questions ----
    if q in [
        "who are you",
        "what are you",
        "what do you do",
        "who is this",
        "what is this system"
    ]:
        return "I am the Mall Operations Assistant for internal management queries."

    # Step 1: If query is NOT database-related,
    # respond directly using the chat LLM
    if not is_db_query(user_query):
        return chat_llm(user_query)

    # Step 2: Convert user query into structured intent (JSON)
    intent = query_to_intent(user_query)
    if intent.get("action") == "unknown":
        return "Sorry, I couldn't understand that."

    # Step 3: Execute the intent on Excel data
    result = run_excel_query(intent)

    # Step 4: Evaluate query result
    eval_out = result_evaluator(result)

    # Step 5: If data exists, ask LLM to explain results
    if eval_out["status"] == "ok":
        return chat_llm(format_db_results(user_query, result, intent))

    # Step 6: If no data is found, fallback
    return chat_llm(get_fallback_message(intent, eval_out["reason"]))


In [14]:
# -------------------------------------------------
# Extended test queries to validate:
# - Inventory queries
# - Staff orders
# - Vendor contact & supplier reasoning
# - Natural language (non-DB) handling
#
# While running, manually verify whether
# the answers match expected behavior.
# -------------------------------------------------
tests = [
    # -------- Inventory related --------
    "How many popcorn packets do we have?",  
    # Expected: Quantity from inventory OR vendor suggestion if unavailable

    "Where is Sprite stored?",  
    # Expected: Location from inventory

    # -------- Staff orders --------
    "Show Ayush's staff orders",  
    # Expected: Items + quantities ordered by Ayush

    "What did Rohan order?",  
    # Expected: Staff orders for Rohan OR fallback if none

    # -------- Vendor related (IMPORTANT) --------
    "Who supplies coffee in the mall?",  
    # Expected: Coffee Elite vendor details

    "How can I contact the coffee vendor?",  
    # Expected: Vendor name + contact person + phone/email

    "Give me the email of the coffee supplier",  
    # Expected: arjun@coffeeelite.com

    "Who supplies popcorn and nachos?",  
    # Expected: SnackMaster Inc. details

    "How can I contact the popcorn vendor?",  
    # Expected: Rajesh Kumar + phone/email

    # -------- Cross reasoning (inventory → vendor) --------
    "Popcorn is not available, who should I contact?",  
    # Expected: SnackMaster Inc. contact details

    "If coffee is out of stock, which vendor should I call?",  
    # Expected: Coffee Elite vendor info

    # -------- Non-database query --------
    "Tell me a joke"
    # Expected: Normal chat response (not DB driven)
]

# -------------------------------------------------
# Execute each test query and print results
# Manually inspect answers for correctness
# -------------------------------------------------
for q in tests:
    print("\nQ:", q)
    print("A:", ask(q))



Q: How many popcorn packets do we have?

DEBUG USER QUERY: How many popcorn packets do we have?
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["quantity"], "filters": [{"column": "item_name", "op": "=", "value": "Popcorn"}], "group_by": [], "limit": null}
DEBUG fuzzy 'Popcorn' → 'Caramel Popcorn' (83.07692307692308)
DEBUG normalized: Popcorn → Caramel Popcorn
DEBUG FINAL INTENT: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': '=', 'value': 'Caramel Popcorn'}], 'group_by': [], 'limit': None}
DEBUG running query: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': '=', 'value': 'Caramel Popcorn'}], 'group_by': [], 'limit': None}
DEBUG result rows: 1
DEBUG: Sending to LLM-B
A: We have 30 packets of Caramel Popcorn in inventory.

Q: Where is Sprite stored?

DEBUG USER QUERY: Where is Sprite stored?
DEBUG: Sending to LLM-A

In [15]:
import gradio as gr

# -------------------------------------------------
# Wrapper function for Gradio chat UI
# -------------------------------------------------
def ask_chat(msg, history):
    reply = ask(msg)

    history.append({"role": "user", "content": msg})
    history.append({"role": "assistant", "content": reply})

    return history, ""


# -------------------------------------------------
# Gradio UI definition (Manager-style, text-only)
# -------------------------------------------------
with gr.Blocks() as ui:

    # ===== Header =====
    gr.Markdown("""
    ## 🧭 Mall Operations Manager Dashboard  
    **Internal Control System – Read Only**
    
    **Role:** Mall Manager  
    **Scope:** Inventory • Staff Orders • Machines • Vendors  
    """)

    # ===== Chat Window =====
    chat = gr.Chatbot(
        type="messages",
        height=480,
        label="Operational Query Log"
    )

    # ===== Input Area =====
    box = gr.Textbox(
        label="Manager Query",
        placeholder="Enter an operational query (e.g., stock status, staff orders, vendor contact)"
    )

    btn = gr.Button("Execute Query")

    # ===== Actions =====
    btn.click(
        ask_chat,
        inputs=[box, chat],
        outputs=[chat, box]
    )

print("DEBUG UI ready")
ui.launch()


DEBUG UI ready
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.



DEBUG USER QUERY: hi 
DEBUG: Sending to LLM-B

DEBUG USER QUERY: Where is Sprite stored?
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["location"], "filters": [{"column": "item_name", "op": "=", "value": "Sprite"}], "group_by": [], "limit": null}
DEBUG fuzzy 'Sprite' → 'Sprite' (83.33333333333334)
DEBUG normalized: Sprite → Sprite
DEBUG FINAL INTENT: {'action': 'select', 'table': 'inventory', 'columns': ['location'], 'filters': [{'column': 'item_name', 'op': '=', 'value': 'Sprite'}], 'group_by': [], 'limit': None}
DEBUG running query: {'action': 'select', 'table': 'inventory', 'columns': ['location'], 'filters': [{'column': 'item_name', 'op': '=', 'value': 'Sprite'}], 'group_by': [], 'limit': None}
DEBUG result rows: 1
DEBUG: Sending to LLM-B

DEBUG USER QUERY: How many popcorn packets do we have?
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["quantity"], "filters": [{"column": "item_